# Findings

Plans:
- Test for amount of journeys in cph
- Test for avg of zone-travesal 
- Test for time-specific travels
- Test for station/stop representations
- Test for avg travel time
- 


- $20\%$ of journeys in SJWaypoints are within cph. 
- The average amount of validInternalZones is $13.21$. This is what we expected due to the ticketing system requirement of journeys traversing more than 8 zones must buy a (non-zone) ticket



In [15]:
import pandas as pd
import numpy as np
data = pd.read_csv('../Data/JTOSJW_dec_jan_2022.csv')
data.groupby('JId').agg(list).count()

CreatedOn             35948
SearchStart           35948
SearchEnd             35948
StartStop             35948
EndStop               35948
StartZone             35948
Endzone               35948
internalStartZones    35948
internalValidZones    35948
SJId                  35948
Id                    35948
Name                  35948
Latitude              35948
Longitude             35948
Type                  35948
SJSearchJourney_Id    35948
dtype: int64

In [6]:
#Copenhagen filtering
condition_1_cph = (
    (data['internalValidZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$')
    | # or
    pd.isna(data['internalValidZones']))
    )

condition_2_cph = (
    (data['internalStartZones'].str.match(r'^(1001|1002|1003|1004)$'))
    | # or
    pd.isna(data['internalStartZones'])
    )

In [7]:
data = data[(condition_1_cph)]
data = data[(condition_2_cph)]
data.groupby('JId').agg(list).count()

/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_45468/4143612775.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[(condition_2_cph)]


CreatedOn             7790
SearchStart           7790
SearchEnd             7790
StartStop             7790
EndStop               7790
StartZone             7790
Endzone               7790
internalStartZones    7790
internalValidZones    7790
SJId                  7790
Id                    7790
Name                  7790
Latitude              7790
Longitude             7790
Type                  7790
SJSearchJourney_Id    7790
dtype: int64

In [32]:
# Step 1: Group by 'JId' and aggregate 'internalValidZones' as a list
grouped_data = data.groupby('JId')['internalValidZones'].agg(list)
avg = 0

for SJ_zone in grouped_data:
    avg += len(SJ_zone[0].split(','))

avg / grouped_data.count()



13.21044286191165

In [ ]:
station_counts = data['station_name_column'].value_counts()
